## Scraping module
Scraping of straits times news article using selenium that are returned from the search function (keyword - 'food supply')

### Imports

In [ ]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

In [ ]:
#!pip install newspaper3k

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

from newspaper import Article
import pandas as pd
import datetime

### Newspaper Article Summariser

In [2]:
def getSummary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    return article.summary, article.keywords, article.publish_date

### StraitsTimes Scraper

In [5]:
# change to ur own directory!
driver = webdriver.Chrome("C:/Users/User/Desktop/FYP/chromedriver.exe") 

def getNews():
    output = []
    
    # get news title in a page - need a for loop to get all the pages!
    all_news_per_page=WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME,"queryly_item_title")))
    
    # get news url (i.e. news source) & news summary & keywords using the summariser module
    for i in range(len(all_news_per_page)):
        news_details = []
        xpath = f'//*[@id="resultdata"]/div[{i+2}]/a'        
        url = all_news_per_page[i].find_element_by_xpath(xpath).get_attribute('href')
        summary = getSummary(url)
        date = summary[2].strftime('%d/%m/%Y')[3:]
        news_details.append(date)
        news_details.append(all_news_per_page[i].text)
        news_details.append(url)
        news_details.append(summary[0])
        news_details.append(", ".join(summary[1]))
        output.append(news_details)
    return output

In [6]:
def getStraitsTimesNews(searchTerm):
    base_url = "https://www.straitstimes.com/search?searchkey=" + searchTerm.replace(" ", "%20")
    driver.get(base_url)

    # close ad (if any)
    try:
        iframe=driver.find_elements_by_tag_name("iframe")[3]
        driver.switch_to.frame(iframe)
        driver.find_elements_by_tag_name("iframe")
        ad = driver.switch_to.frame(0)
        driver.find_element_by_id('pclose-btn').click()
        driver.switch_to.default_content()
    except:
        driver.switch_to.default_content()
    
    # filter the search result by date by xpath accordingly
    # within 24 hours, within one week, within 30 days, within a year, within three years
    filter_by_date = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.XPATH,"//*[contains(text(), 'within three years')]")))[0].click()

    # get news on page
    output = getNews()
    
    # check for next page
    nextpage = True
    while (nextpage):  
        try:
            nextpage = WebDriverWait(driver,20).until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR,'#resultdata > a > h2')))[0]
            if (nextpage.text == 'Next Page'):
                nextpage.click()
#                 print('clicked next')
                output.extend(getNews())
            else: 
                nextpage = False
        except:
#             print("no next page!")
            nextpage = False
    
    
    # convert to df and export as csv
    df = pd.DataFrame(output, columns=['date','news title', 'news source(url)', 'content summary', 'keywords'])
    df.to_csv(f"csv_data/{searchTerm}_straitstimes.csv")
    return df

getStraitsTimesNews("food supply")

,date,news title,news source(url),content summary,keywords
0,02/2022,Hong Kong food supplies may be disrupted as Co...,https://www.straitstimes.com/asia/east-asia/ho...,HONG KONG (REUTERS) - Hong Kong authorities sa...,"chinese, supply, supplies, kong, covid19, good..."
1,02/2022,Hong Kong food supply at risk again as more dr...,https://www.straitstimes.com/asia/east-asia/ho...,All places visited by the two drivers were dis...,"china, supply, kong, vegetables, covid19, fres..."
2,02/2022,First batch of Temasek rice grown in the commu...,https://www.straitstimes.com/singapore/environ...,SINGAPORE - The made-in-Singapore Temasek Rice...,"farm, water, rice, tampines, grown, batch, gro..."
3,02/2022,Hong Kong residents raid supermarket shelves a...,https://www.straitstimes.com/asia/east-asia/ho...,HONG KONG (REUTERS) - Hong Kong residents crow...,"supermarket, supply, supplies, kong, raid, cov..."
4,02/2022,Arctic seed vault to receive rare deposits,https://www.straitstimes.com/world/arctic-seed...,OSLO (REUTERS) - A vault built on an Arctic mo...,"receive, banks, war, syria, arctic, deposits, ..."
5,02/2022,"Hong Kong reports 1,347 Covid-19 cases as heal...",https://www.straitstimes.com/asia/east-asia/ho...,"HONG KONG (REUTERS, BLOOMBERG) - Hong Kong rep...","overwhelmed, kong, infections, covid19, health..."
6,02/2022,"Measles cases, deaths surging in Afghanistan: WHO",https://www.straitstimes.com/asia/measles-case...,"""Measles cases have been increasing in all pro...","vaccination, vulnerable, deaths, cases, surgin..."
7,02/2022,Young people continue to make up majority of f...,https://www.straitstimes.com/singapore/courts-...,SINGAPORE - Young people continue to make up t...,"offenders, 2020, 30, bureau, firsttime, old, d..."
8,02/2022,Shoppers throng Hong Kong's markets after Covi...,https://www.straitstimes.com/asia/east-asia/sh...,HONG KONG (AFP) - Shoppers thronged Hong Kong'...,"supply, vegetable, siu, covid19, hong, throng,..."
9,02/2022,US posts record trade deficit in 2021 on surgi...,https://www.straitstimes.com/business/economy/...,WASHINGTON (REUTERS) - The US trade deficit su...,"imports, record, deficit, 2021, billion, goods..."
